In [ ]:
import pandas as pd
import numpy as np
import warnings
from functions import metrics
import csv
from sklearn.exceptions import ConvergenceWarning
from functions.formatting import get_subgroup_str
from itertools import product, combinations
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
omit_demographics = True 
use_gerryfair = False 

In [ ]:
df = pd.read_csv('./data/preprocessed.csv')
df.drop(['umich_user_id', 'Unnamed: 0'], axis=1, inplace=True)

In [ ]:
X = df.drop('completed', axis=1)
y = df['completed']

#### Calculate false positive rates for the given subgroup

This function takes in the subgroup to test for as a list of tuples. We use logistic regression and 10-fold cross validation to calculate the average FPR and AUC for the given subgroup. 

Returns a tuple containing FPR average, FPR standard deviation, AUC average, AUC standard deviation, and number of students in a given subgroup

#### Compute all subgroups

This function generates all possible subgroups given the list of protected columns

In [ ]:
demo_groups = [['US', 'intl'], ['bachelor_obtained', 'no_bachelor_obtained', 'education_na'], ['white', 'black', 'asian', 'latinx', 'race_others', 'race_na'], ['male', 'female', 'gender_na', 'gender_other']]

In [ ]:
def compute_combos():
    ret = []
    for i in range(1,len(demo_groups)+1):
        g = list(combinations(demo_groups, i))
        for demos in g:
            ret += list(product(*demos))

    return ret

In [ ]:
combos = compute_combos()

In [ ]:
combos.sort(key=len)
# combos
combos.insert(0, ())

In [ ]:
combos

In [ ]:
demographics = ['US', 'intl', 'bachelor_obtained', 'no_bachelor_obtained', 'education_na', 'white', 'black', 'asian', 'latinx', 'race_others', 'race_na', 'male', 'female', 'gender_na', 'gender_other']
protected=['US', 'intl', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female']

In [ ]:
res = metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=False, iters=3)

In [ ]:
# res

Generate a list of all subgroup data and write it to a csv

In [ ]:
csv_file = f"./data/MTC508_subgroup_data_without_demographics.csv"
# csv_file = "test.csv"

# Write the data to a CSV file
with open(csv_file, 'w', newline='') as csvfile:
    fieldnames = [
        'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()  # Write the header row
    for row in res:
        writer.writerow(row)

In [ ]:
# data = {}

# data['overall'] = len(X)

# for c in combos[1:]:
#     masks = [X[name] == value for name, value in c]
#     final_mask = pd.concat(masks, axis=1).all(axis=1)
#     filter_X = X[final_mask]
#     data[get_subgroup_str(c)] = len(filter_X)




In [ ]:
# df = pd.DataFrame(list(data.items()), columns=['Key', 'Count'])

# # Specify the CSV file path
# csv_file_path = 'data/MTC508_data_counts.csv'

# # Write the DataFrame to a CSV file
# df.to_csv(csv_file_path, index=False)

In [ ]:
subgroup_data = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')
fpr_data = {}
# loop through each subgroup
# for each subgroup, get the corresponding data
for index, row in subgroup_data.iterrows():
    # print(f"Index: {index}, fpr: {row['fpr_avg']}")
    fpr_data[row['subgroup']] = row['fpr_avg']



In [ ]:
subgroup_data = pd.read_csv('./data/MTC508_subgroup_data_without_demographics.csv')
auc_data = {}
# loop through each subgroup
# for each subgroup, get the corresponding data
for index, row in subgroup_data.iterrows():
    # print(f"Index: {index}, fpr: {row['fpr_avg']}")
    auc_data[row['subgroup']] = row['auc_avg']

In [ ]:
def get_data_iters(start, stop, step, attrs, protected):
    ret_val_auc = []
    ret_val_fpr = []
    data = []
    for i in range(start+1,stop+1,step):
        data.append(metrics.calc_metrics(X, y, combos, omit_demographics=True, demographics=demographics, protected=protected, is_gerryfair=True, iters=i))
    
    for a in attrs:
        aucs = []
        fprs = []
        for res in data:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    aucs.append(float(subgroup['auc_avg']))
                    fprs.append(float(subgroup['fpr_avg']))
                    break

        iterations = np.array(list(range(start, stop, step)))

        other_algorithm_auc = np.linspace(auc_data[a], auc_data[a], iterations.size)
        other_algorithm_fpr = np.linspace(fpr_data[a], fpr_data[a], iterations.size)

        ret_val_auc.append((iterations, aucs, other_algorithm_auc))
        ret_val_fpr.append((iterations, fprs, other_algorithm_fpr))
    
    return ret_val_auc, ret_val_fpr
        


In [ ]:
attrs = ['Overall', 'white', 'black', 'female', 'black, female']

In [ ]:
val_auc, val_fpr = get_data_iters(0, 301, 20, attrs, ['white', 'black', 'asian', 'latinx'])

In [ ]:
val

In [ ]:
for i, g in enumerate(attrs):
    iterations, fprs, other_algorithm_fpr = val[i]

    df = pd.DataFrame({'x_axis': iterations, 'y_axis': fprs, 'other': other_algorithm_fpr})
    df.to_csv('./data/graph_iters_auc_0_301_20_' + g + '.csv')

    

In [ ]:
val = []
for i, g in enumerate(attrs):
    data = pd.read_csv('./data/graph_iters_fpr_0_1001_20' + g + '.csv')
    line = [list(data['x_axis']), list(data['y_axis']), list(data['other'])]

    val.append(line)
    

In [ ]:
val

In [ ]:
def build_graph(x_axis, y_axis, other_algorithm_fpr, x_axis_label, y_axis_label, a):
        plt.plot(x_axis, y_axis, linestyle='-', label='GerryFair')

        plt.plot(x_axis, other_algorithm_fpr, linestyle='-', label='LR Without Gerryfair')
        plt.ylim(0, 1)

        plt.xlabel(x_axis_label)
        plt.ylabel(y_axis_label)
        plt.title(f'Comparison of Algorithms - {y_axis_label} vs {x_axis_label} For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
attributes = ["Overall", "White", "Black", "Female", "Black + Female"]

for i, attr in enumerate(attributes):
    build_graph(val[i][0], val[i][1], val[i][2], 'Iterations', 'FPR', attr)

In [ ]:
import matplotlib.pyplot as plt

def build_combined_graph(val, attributes):
    # Set distinct colors for each attribute
    colors = ['b', 'g', 'r', 'c', 'm']  # You can add more colors if needed

    # Create a single plot
    plt.figure(figsize=(12, 6))  # Adjust the figure size as needed

    for i, attr in enumerate(attributes):
        x_axis, y_axis, other_algorithm_fpr = val[i]
        plt.plot(x_axis, y_axis, linestyle='-', label=f'GerryFair - {attr.capitalize()}', color=colors[i])
        plt.plot(x_axis, other_algorithm_fpr, linestyle='-', label=f'LR Without Gerryfair - {attr.capitalize()}', color=colors[i], alpha=0.5)

    # Set the y-axis range to 0 to 1
    plt.ylim(0, 1)

    plt.xlabel('Iterations')
    plt.ylabel('FPR')
    plt.title('Comparison of Algorithms - AUC vs Iterations (All Protected)')

    # Move the legend outside the axes
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Automatically adjust subplot parameters to give specified padding
    plt.tight_layout()

    plt.show()

attributes = ["Overall", "White", "Black", "Female", "Black + Female"]

build_combined_graph(val, attributes)


In [ ]:
def build_graph_gamma(iters, gammas, attrs):
    results = []
    for g in gammas:
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=['country_cd_US', 'bachelor_obtained', 'white', 'black', 'asian', 'latinx', 'male', 'female'], is_gerryfair=True, iters=iters+1, gamma=g))
    
    data = results
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        gammas = np.array(gammas)

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], gammas.size) 

        plt.plot(gammas, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(gammas, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Gamma')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Gamma For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
build_graph_gamma(5, [.002, .005, .01, .02, .05, .1], ['Overall', 'black', 'female', 'black, female'])

In [ ]:
len(data)

In [ ]:
def gen_data_attrs(iters, attrs, sets):
    results = []
    for i in range(len(sets)):
        results.append(metrics.calc_metrics(X, y, groups, omit_demographics=not include_demographics, demographics=demographics[:i+1], is_gerryfair=True, iters=iters+1, gamma=g))
    
    for a in attrs:
        fprs = []
        for res in results:
            for subgroup in res:
                if subgroup['subgroup'] == a:
                    fprs.append(float(subgroup['fpr_avg']))
                    break
        
        fprs = np.array(fprs)
        num_attrs = np.array(list(range(1,len(demographics) + 1)))

        other_algorithm_fpr = np.linspace(default_fprs[a], default_fprs[a], len(demographics)) 

        plt.plot(num_attrs, fprs, marker='o', linestyle='-', label='GerryFair')

        plt.plot(num_attrs, other_algorithm_fpr, marker='s', linestyle='-', label='Other Algorithm')

        plt.xlabel('Number of Attributes')
        plt.ylabel('False Positive Rate (FPR)')
        plt.title('Comparison of Algorithms - FPR vs Number of Attributes For ' + a.capitalize())

        plt.legend()

        plt.show()

In [ ]:
gerryfair_groups = {
    'all': protected,
    'location': demo_groups[0], 
    'education': demo_groups[1][:-1], 
    'race': demo_groups[2][:-2], 
    'gender': demo_groups[3][:-2],
    'race+gender': demo_groups[2][:-2] + demo_groups[3][:-2]
    }

In [ ]:
def gen_data_categories(iters, attrs):
    for g in attrs:
        res = metrics.calc_metrics(X, y, combos, omit_demographics=False, demographics=demographics, protected=attrs[g], is_gerryfair=True, iters=iters)
        csv_file = f"./data/graph_attrs_{str(iters)}_{g}.csv"

        # Write the data to a CSV file
        with open(csv_file, 'w', newline='') as csvfile:
            fieldnames = [
                'subgroup', 'n', 'auc_avg', 'auc_std', 'fpr_avg', 'fpr_std', 'rmse_avg', 'rmse_std']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            writer.writeheader()  # Write the header row
            for row in res:
                writer.writerow(row)

In [ ]:
gen_data_categories(200, gerryfair_groups)

In [ ]:
demographics

In [ ]:
demo_groups[2][-2]

In [ ]:
protected